In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import cv2
import numpy as np
from glob import glob

In [ ]:
CLASS_NAME_DICT =  dict(
              ((40,  'TURN_RESTRICTION_US_RIGHT'),
               (41,  'TURN_RESTRICTION_US_UTURN'),
               (43,  'SL_STOP_SIGN'),
               (123, 'TRAFFIC_LIGHTS_SIGN'),
               (39,  'TURN_RESTRICTION_US_LEFT'),
               (16,  'SL_US_5'), 
               (17,  'SL_US_10'), 
               (18,  'SL_US_15'), 
               (19,  'SL_US_20'), 
               (20,  'SL_US_25'),
               (21,  'SL_US_30'), 
               (22,  'SL_US_35'), 
               (23,  'SL_US_40'), 
               (24,  'SL_US_45'), 
               (25,  'SL_US_50'),
               (26,  'SL_US_55'), 
               (27,  'SL_US_60'), 
               (28,  'SL_US_65'), 
               (29,  'SL_US_70'), 
               (30,  'SL_US_75'),
               (31,  'SL_US_80'),
               (42,  'TURN_RESTRICTION_US_LEFT_UTURN'),
               (69,  'GIVE_WAY')))

def read_df(metadata):

    images = metadata.images    
    data_df = pd.DataFrame({"img_proto": images})
    data_df.loc[:,"img_name"] = data_df.loc[:,"img_proto"].apply(lambda image: image.metadata.image_path)
    data_df.loc[:, "rois"] = data_df.loc[:,"img_proto"].apply(lambda image:image.rois)
    data_df.loc[:, "nr_rois"] = data_df.loc[:,"rois"].apply(len)
    data_df = data_df.drop(["img_proto"],axis=1)
        
    return data_df

def explode_rois(data_df):
    
    rois_df_cols = list(data_df.columns)
    rois_df_cols.append("roi")

    rows = []
    _ = data_df.apply(lambda row: [rows.append(row.tolist() + [nn]) for nn in row.rois], axis=1)

    rois_df = pd.DataFrame(rows, columns=rois_df_cols)
    
    rois_df.loc[:,"roi_type"] = rois_df.loc[:,"roi"].apply(lambda roi: CLASS_NAME_DICT[roi.type])
    
    rois_df = rois_df.drop(["rois","nr_rois"],axis=1)
    
    return rois_df

def plot_data_df(data_df,nr_images_to_plot):
    for _,row in data_df.iloc[:nr_images_to_plot].iterrows():

        img_path = os.path.join(images_folder,row["img_name"])
        img = np.asarray(PIL.Image.open(img_path))

        gt_image_rois = row["rois"]

        for roi in gt_image_rois:
            tl_col = roi.rect.tl.col
            tl_row = roi.rect.tl.row
            br_col = roi.rect.br.col
            br_row = roi.rect.br.row

            img = cv2.rectangle(img, (tl_col,tl_row), (br_col, br_row), (0,255,0), 10)
            img = cv2.putText(img, CLASS_NAME_DICT[roi.type], (tl_col-10, tl_row-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 5)


        plt.figure(figsize=(15,15))
        plt.imshow(img)
        plt.show()
        
        
def plot_roi_df(roi_df,nr_rois_to_plot):
    for _,row in roi_df.iloc[:nr_rois_to_plot].iterrows():

        img_path = os.path.join(images_folder,row["img_name"])
        img = np.asarray(PIL.Image.open(img_path))

        roi = row["roi"]

        tl_col = roi.rect.tl.col
        tl_row = roi.rect.tl.row
        br_col = roi.rect.br.col
        br_row = roi.rect.br.row

        img = cv2.rectangle(img, (tl_col,tl_row), (br_col, br_row), (0,255,0), 10)
        img = cv2.putText(img, CLASS_NAME_DICT[roi.type], (tl_col-10, tl_row-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 5)


        plt.figure(figsize=(15,15))
        plt.imshow(img)
        plt.show()
        

# 1.Setup Environment

Here we will download the zip containing the dataset and then perform some basic manipulations on the images and on the ROIs (region of interest). Below, we will refer to all the labeled ground truth as ROIs

This notebook will only work on a Unix-based environment

## 1.1 Download zip

In [ ]:
!wget : https://s3.eu-central-1.amazonaws.com/telenav.ai/telenav_ai_dataset_sample.zip

## 1.2 Unzip zip file

In [ ]:
!unzip telenav_ai_dataset_sample.zip

## 1.3 Import necessary proto files

When reading the protobuf files, the path to the generated protobuf definitions located in the **protobuf** folder needs to be in your python path.

You can also check the structure of the protobuf files in the **protobuf** folder

In [ ]:
sys.path.append("./telenav_ai_dataset_sample/protobuf")
import proto_api as proto_api

## 1.4 Set Paths

In [ ]:
images_folder = "./telenav_ai_dataset_sample/sample_data/"
rois_path = os.path.join(images_folder,"rois.bin")
images_paths = glob(images_folder + "*[jpg,jpeg]")

# 2 Read Data

In [ ]:
imageset_proto = proto_api.read_metadata(rois_path)
data_df = read_df(imageset_proto)
data_df.head()

## 2.1 Basic Stats

In [ ]:
print("Number of images: {}".format(len(data_df)))
print("Number of rois: {}".format(data_df["nr_rois"].sum()))
print("Count by number of rois in image: ")
data_df["nr_rois"].value_counts()

In [ ]:
rois_df = explode_rois(data_df)

print("Number of ROIs by type: ")
rois_df["roi_type"].value_counts()


## 2.2 Visualize Images with all ROIs

In [ ]:
non_empty_df = data_df[data_df["nr_rois"] != 0]

In [ ]:
plot_data_df(non_empty_df,nr_images_to_plot = 10)

## 2.3 Visualize ROIs of specific type

Note that this visualization is a bit different from the one above, in the sense that each ROI is plotted in a separate figure, even if they are part of the same image. In the visualization above, we plotted an image with all of its ROIs in a single figure

In [ ]:
specific_type_rois_df = rois_df[rois_df["roi_type"] == "TURN_RESTRICTION_US_RIGHT"]

In [ ]:
plot_roi_df(specific_type_rois_df,nr_rois_to_plot = 10)